In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn import preprocessing, svm, dataset
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import seaborn as sns
import math
#evaluation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

f = open("breast-cancer-wisconsin.data", 'r')
list = []
while True:
    line = f.readline()[:-1]
    if not line: break
    list.append(line.split(','))
f.close()
#.data 파일 읽기

df = pd.DataFrame(list)
df
df = df.apply(pd.to_numeric, errors = 'coerce')
# numeric 안되는 것은 nan
df = df.dropna()
data = df.iloc[:,1:10]
target = df.iloc[:,10]
#결측치 제거 후 data, target으로 분할

# print(data)
# print(target)

scaler = StandardScaler()
ss_data = scaler.fit_transform(data)
ss_data = pd.DataFrame(ss_data)
# 표준화된 feature

print(ss_data)
print(target)

df.plot(y=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
plt.show()

train_data, test_data, train_target, test_target = train_test_split(data, target)
ss_train_data, ss_test_data, ss_train_target, ss_test_target = train_test_split(ss_data, target)
#표준화 한 데이터, 하지 않은 데이터 준비

#k-fold
def kfoldValidation(model, X, y, k):
    kfold = KFold(n_splits = k, shuffle = True, random_state = 0)
    scores = cross_val_score(model, X, y, cv = kfold)
    return scores

#DecisionTree
def dt(criterion, X_train, Y_train, X_test, Y_test):
    tree = DecisionTreeClassifier(criterion = criterion)
    params = {'max_depth':[2, 3, 4, 5], 'min_samples_split': [2, 3, 4]}
    grid_tree = GridSearchCV(tree, param_grid=params)
    grid_tree.fit(X_train, Y_train)
    print(grid_tree.best_params_)
    print(grid_tree.score(X_test, Y_test))

# logistic regression
def lr(X_train, Y_train, X_test, Y_test):
    logi = LogisticRegression()
    params = {'C':[0.01, 0.1, 1, 10, 100]}
    grid_logi = GridSearchCV(logi, param_grid=params)
    grid_logi.fit(X_train, Y_train)
    print(grid_logi.best_params_)
    print(grid_logi.score(X_test, Y_test))

#'kernel': ['poly', 'rbf', 'sigmoid'], 
def svm(X_train, Y_train, X_test, Y_test):
    params = {'C': [0.01, 0.1, 1, 10, 100], 'gamma': [0.7, 0.5, 0.3, 0.1, 0.05]}
    svm_poly = SVC(kernel = 'poly')
    svm_rbf = SVC(kernel = 'rbf')
    svm_sig = SVC(kernel = 'sig')
    
    grid_poly = GridSearchCV(svm_poly, param_grid=params)
    grid_rbf = GridSearchCV(svm_rbf, param_grid=params)
    grid_sig = GridSearchCV(svm_sig, param_grid=params)
    grid_poly.fit(X_train, Y_train)
    grid_rbf.fit(X_train, Y_train)
    print("poly", grid_poly.best_params_)
    print(grid_poly.score(X_test, Y_test))
    print("rbf",grid_rbf.best_params_)
    print(grid_rbf.score(X_test, Y_test))

#without Scaler
def runModel(train_data, train_target, test_data, test_target):
    print("DecisionTree-entropy")
    dt('entropy', train_data, train_target, test_data, test_target)

    print("DecisionTree-gini")
    dt('gini', train_data, train_target, test_data, test_target)

    print("logistic regression")
    lr(train_data, train_target, test_data, test_target)
    
    print("SVM")
    svm(train_data, train_target, test_data, test_target)

#with Scaler
print("without Scaler")
runModel(train_data, train_target, test_data, test_target)
print("with Scaler")
runModel(ss_train_data, ss_train_target, ss_test_data, ss_test_target)

#confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier

#dt_clf = DecisionTreeClassifier(random_state=10)
#dt_clf.fit(train_data, train_target)

lr_clf = LogisticRegression()
lr_clf.fit(train_data, train_target)

#svm_clf = ()
#svm_clf.fit(train_data, train_target)

print("--Confusion matrix of DecisionTree--")
#confusion_matrix(train_target, dt_clf.predict(train_data))

print("--Confusion matrix of LogisticRegression--")
print(confusion_matrix(train_target, lr_clf.predict(train_data)))

print("--Confusion matrix of SVM--")
#confusion_matrix(train_target, svm_clf.predict(train_data))


